In [ ]:
from src.Models.SpectralTransformer import SpectralTransformer
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import torch


_, test_loader = get_dataloaders(self.rawDataDirectory, self.remasteredDataDirectory, args.test_batch_size)

model = SpectralTransformer().to(device)
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

psnr_values = []
ssim_values = []

results_dir = 'evaluation_results'
os.makedirs(results_dir, exist_ok=True)

with torch.no_grad():
    for i, (raw_img, ref_img) in enumerate(test_loader):
        raw_img = raw_img.to(device)
        ref_img = ref_img.to(device)

        enhanced_img = model(raw_img)

        enhanced_np = enhanced_img.cpu().squeeze().permute(1, 2, 0).numpy()
        enhanced_np = np.clip(enhanced_np, 0, 1)  # Clip values to [0, 1]

        ref_np = ref_img.cpu().squeeze().permute(1, 2, 0).numpy()
        raw_np = raw_img.cpu().squeeze().permute(1, 2, 0).numpy()

        curr_psnr = psnr(ref_np, enhanced_np)
        curr_ssim = ssim(ref_np, enhanced_np, multichannel=True)
        psnr_values.append(curr_psnr)
        ssim_values.append(curr_ssim)

        # Print progress
        if (i + 1) % 20 == 0:
            print(f"Processed {i + 1}/{len(test_loader)} test images")

avg_psnr = np.mean(psnr_values)
avg_ssim = np.mean(ssim_values)

print(f"Evaluation Results:")
print(f"Average PSNR: {avg_psnr:.2f} dB")
print(f"Average SSIM: {avg_ssim:.4f}")

with open(f'{results_dir}/metrics.txt', 'w') as f:
    f.write(f"Average PSNR: {avg_psnr:.2f} dB\n")
    f.write(f"Average SSIM: {avg_ssim:.4f}\n")

In [ ]:
from src.Models.SpectralTransformer import SpectralTransformer

model = SpectralTransformer()

